In [2]:
# import libraries for modeling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import random

import os
import cv2
import glob
from skimage.color import rgb2gray

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [18]:
n = 0
xt = np.load('../data/final/3d/xtrain_40-%d.npy' % (n))
yt = np.load('../data/final/3d/ytrain_40-0.npy')

In [19]:
print(xt.shape)
print(yt.shape)

(2305, 64, 64, 3)
(2305,)


In [20]:
xt = xt[:100, :, :, 0]
yt = yt[:100]

In [29]:
np.stack((xt[x_nor[0][0], :, :], xt[x_nor[0][1], :, :]), axis=2)[:, :, 0]

array([[ 71,  68,  63, ..., 178, 178, 178],
       [ 79,  75,  68, ..., 178, 178, 178],
       [ 92,  86,  77, ..., 179, 179, 179],
       ...,
       [ 87,  79,  66, ..., 161, 163, 164],
       [ 90,  81,  64, ..., 162, 165, 166],
       [ 92,  81,  63, ..., 164, 166, 168]], dtype=uint8)

In [30]:
np.stack((xt[x_nor[0][0], :, :], xt[x_nor[0][1], :, :]), axis=2)[:, :, 1]

array([[139, 149, 145, ..., 164, 168, 170],
       [134, 147, 148, ..., 150, 161, 168],
       [124, 141, 133, ..., 130, 146, 161],
       ...,
       [139, 108, 104, ..., 102, 103, 103],
       [140, 106,  96, ..., 102, 104, 105],
       [127, 118,  89, ..., 103, 105, 106]], dtype=uint8)

In [53]:
roi_2c[0, :, :, 0]

array([[ 71,  68,  63, ..., 178, 178, 178],
       [ 79,  75,  68, ..., 178, 178, 178],
       [ 92,  86,  77, ..., 179, 179, 179],
       ...,
       [ 87,  79,  66, ..., 161, 163, 164],
       [ 90,  81,  64, ..., 162, 165, 166],
       [ 92,  81,  63, ..., 164, 166, 168]], dtype=uint8)

In [54]:
roi_2c[0, :, :, 1]

array([[139, 149, 145, ..., 164, 168, 170],
       [134, 147, 148, ..., 150, 161, 168],
       [124, 141, 133, ..., 130, 146, 161],
       ...,
       [139, 108, 104, ..., 102, 103, 103],
       [140, 106,  96, ..., 102, 104, 105],
       [127, 118,  89, ..., 103, 105, 106]], dtype=uint8)

In [37]:
len(x_nor[0])
len(x_ede[0])

31

In [39]:
roi_2c = np.stack((xt[x_nor[0][0], :, :], xt[x_nor[0][i], :, :]), axis=2)
roi_2c = roi_2c.ravel()
roi_2c = roi_2c.reshape(-1, 64, 64, 2)
print(roi_2c.shape)

IndexError: index 69 is out of bounds for axis 0 with size 69

In [52]:
x_nor = np.where(yt==0)
x_ede = np.where(yt==1)

c = 0
for i in range(1, len(x_nor[0])):
    if x_nor[0][i-1] + 3 > x_nor[0][i]:
        if c == 0:
            roi_2c = np.stack((xt[x_nor[0][i-1], :, :], xt[x_nor[0][i], :, :]), axis=2)
            roi_2c = roi_2c.ravel()
            roi_2c = roi_2c.reshape(-1, 64, 64, 2)
            print(roi_2c.shape)
        else:
            a = np.stack((xt[x_nor[0][i-1], :, :], xt[x_nor[0][i], :, :]), axis=2)
            a = a.ravel()
            a = a.reshape(-1, 64, 64, 2)
            roi_2c = np.vstack((roi_2c, a))
    c += 1   
#             print(roi_2c.shape)

print(roi_2c.shape)

(1, 64, 64, 2)
(62, 64, 64, 2)


In [41]:
print(c)

62


In [ ]:
## a = []
b = 0
for j in [1]:
    if j < 10:
        j = '0' + str(j) + '/'
    else:
        j = str(j) + '/'

    path = '/media/miruware/data/train50_final/' + j
    

    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.jpg'):
                img = cv2.imread(path + filename)
                    
                file_name, file_ext = os.path.splitext(filename)
                    
                with open(path + file_name + '.hytag', 'r') as f:
                    a.append(len(f.readlines()))
                    b += 1
                with open(path + file_name + '.hytag', 'r') as f:
                    while True:                
                        line = f.readline()
                        if line:
                            line = line.strip().split(',')
                            
                            
# # #                                 ROI coordinate extraction
#                             hytag_list = line 
#                             x=int(hytag_list[4]); y=int(hytag_list[5]); w=int(hytag_list[6]); h=int(hytag_list[7]) 
#                             print(x)
                        else:
                            break

## Create Train(40) and Valid(10) Dataset

In [49]:
# data preprocessing 
for n in range(10):
    random.seed(n)
    l = [i for i in range(1, 51)]
    random.shuffle(l)
#     print(l[:2])
    c = 0
    c2 = 0
    
    # Made Simple Train Set
    # folder and file path
    # 40개 case를 trainset 으로 저장.
    for j in l[:40]:
        labels = np.zeros(5000)  # store labels
        ls_counts = 0
        
        if j < 10:
            j = '0' + str(j) + '/'
        else:
            j = str(j) + '/'
        
        path = '/media/miruware/data/train50_final/' + j
        
        
        for dirname, _, filenames in os.walk(path):
            filenames = sorted(filenames)
            for filename in filenames:
#                 print(filename)
                if filename.endswith('.jpg'):
                    img = cv2.imread(path + filename)
                    
                    file_name, file_ext = os.path.splitext(filename)
                    

                    with open(path + file_name + '.hytag', 'r') as f:

                        while True:                
                            line = f.readline()
                            if line:
                                line = line.strip().split(',')
                            
                                # ROI coordinate extraction
                                hytag_list = line 
                                x=int(hytag_list[4]); y=int(hytag_list[5]); w=int(hytag_list[6]); h=int(hytag_list[7]) 
                            
                                # label extraction
                                
                                if hytag_list[0] == 'LBUID_636802211114365653_0529': # Edema
                                    labels[ls_counts] = 1
                                elif hytag_list[0] == 'LBUID_636802210898995628_5475': # Normal 
                                    labels[ls_counts] = 0

                                # Resize image(Minimal size for xline and yline)
                                roi = img[y:y+h, x:x+w]
                                
                                roi_resize = cv2.resize(roi, (64, 64))
                                roi_resize_ravel = roi_resize.ravel()
                                roi_resize_ravel = roi_resize_ravel.reshape(-1, 64, 64, 3)

#                                 roi_resize_ravel = rgb2gray(roi_resize_ravel)

                                ls_counts += 1

                                # stack arrays for making trainset
                                if ls_counts < 2:
                                    roi_all = roi_resize_ravel
                                else:
                                    roi_all = np.vstack((roi_all, roi_resize_ravel))
#                                 print(roi_all.shape)
                            else:
                                break
                        
# #                 else:
# #                     pass

        roi_all = roi_all[:, :, :, 0]

        xt = roi_all
        labels = labels[:roi_all.shape[0]]
#         print(labels)

        x_nor = np.where(labels==0)
        x_ede = np.where(labels==1)
#         print(x_ede[0])

        for i in range(1, len(x_nor[0])):
            if x_nor[0][i-1] + 3 > x_nor[0][i]:
                if c == 0:
                    roi_2c = np.stack((xt[x_nor[0][i-1], :, :], xt[x_nor[0][i], :, :]), axis=2)
                    roi_2c = roi_2c.ravel()
                    roi_2c = roi_2c.reshape(-1, 64, 64, 2)
#                     print(roi_2c.shape)


                else:
                    a = np.stack((xt[x_nor[0][i-1], :, :], xt[x_nor[0][i], :, :]), axis=2)
                    a = a.ravel()
                    a = a.reshape(-1, 64, 64, 2)
                    roi_2c = np.vstack((roi_2c, a))
#                     print(roi_2c.shape)

                c += 1  

        
            
        for i in range(1, len(x_ede[0])):
            if x_ede[0][i-1] + 3 > x_ede[0][i]:
                if c2 == 0:
#                     print(i)
                    roi_2c_ede = np.stack((xt[x_ede[0][i-1], :, :], xt[x_ede[0][i], :, :]), axis=2)
                    roi_2c_ede = roi_2c_ede.ravel()
                    roi_2c_ede = roi_2c_ede.reshape(-1, 64, 64, 2)
#                     print(roi_2c_ede.shape)

                else:
#                     print(i)
                    a = np.stack((xt[x_ede[0][i-1], :, :], xt[x_ede[0][i], :, :]), axis=2)
                    a = a.ravel()
                    a = a.reshape(-1, 64, 64, 2)
                    roi_2c_ede = np.vstack((roi_2c_ede, a))
#                     print(roi_2c_ede.shape)
                c2 += 1
    
    label_train_nor = np.zeros(roi_2c.shape[0])
    label_train_ede = np.ones(roi_2c_ede.shape[0])
    
    roi_2c_train = np.vstack((roi_2c, roi_2c_ede))
    label_train = np.hstack((label_train_nor, label_train_ede))

#     print(roi_2c_train.shape)
#     print(label_train.shape)

    
    c = 0
    c2 = 0
#     ls_counts = 0
#     labels2 = np.zeros(5000)  # store labels
    
    for j in l[40:]:
        labels = np.zeros(5000)  # store labels
        ls_counts = 0
        
        if j < 10:
            j = '0' + str(j) + '/'
        else:
            j = str(j) + '/'
        
        path = '/media/miruware/data/train50_final/' + j
        
        
        for dirname, _, filenames in os.walk(path):
            filenames = sorted(filenames)
            for filename in filenames:
#                 print(filename)
                if filename.endswith('.jpg'):
                    img = cv2.imread(path + filename)
                    
                    file_name, file_ext = os.path.splitext(filename)
                    

                    with open(path + file_name + '.hytag', 'r') as f:

                        while True:                
                            line = f.readline()
                            if line:
                                line = line.strip().split(',')
                            
                                # ROI coordinate extraction
                                hytag_list = line 
                                x=int(hytag_list[4]); y=int(hytag_list[5]); w=int(hytag_list[6]); h=int(hytag_list[7]) 
                            
                                # label extraction
                                
                                if hytag_list[0] == 'LBUID_636802211114365653_0529': # Edema
                                    labels[ls_counts] = 1
                                elif hytag_list[0] == 'LBUID_636802210898995628_5475': # Normal 
                                    labels[ls_counts] = 0

                                # Resize image(Minimal size for xline and yline)
                                roi = img[y:y+h, x:x+w]
                                
                                roi_resize = cv2.resize(roi, (64, 64))
                                roi_resize_ravel = roi_resize.ravel()
                                roi_resize_ravel = roi_resize_ravel.reshape(-1, 64, 64, 3)

#                                 roi_resize_ravel = rgb2gray(roi_resize_ravel)

                                ls_counts += 1

                                # stack arrays for making trainset
                                if ls_counts < 2:
                                    roi_all = roi_resize_ravel
                                else:
                                    roi_all = np.vstack((roi_all, roi_resize_ravel))
#                                 print(roi_all.shape)
                            else:
                                break
                        
# #                 else:
# #                     pass

        roi_all = roi_all[:, :, :, 0]

        xt = roi_all
        labels = labels[:roi_all.shape[0]]
#         print(labels)

        x_nor = np.where(labels==0)
        x_ede = np.where(labels==1)
#         print(x_ede[0])

        for i in range(1, len(x_nor[0])):
            if x_nor[0][i-1] + 3 > x_nor[0][i]:
                if c == 0:
                    roi_2c = np.stack((xt[x_nor[0][i-1], :, :], xt[x_nor[0][i], :, :]), axis=2)
                    roi_2c = roi_2c.ravel()
                    roi_2c = roi_2c.reshape(-1, 64, 64, 2)
#                     print(roi_2c.shape)


                else:
                    a = np.stack((xt[x_nor[0][i-1], :, :], xt[x_nor[0][i], :, :]), axis=2)
                    a = a.ravel()
                    a = a.reshape(-1, 64, 64, 2)
                    roi_2c = np.vstack((roi_2c, a))
#                     print(roi_2c.shape)

                c += 1  

        
            
        for i in range(1, len(x_ede[0])):
            if x_ede[0][i-1] + 3 > x_ede[0][i]:
                if c2 == 0:
#                     print(i)
                    roi_2c_ede = np.stack((xt[x_ede[0][i-1], :, :], xt[x_ede[0][i], :, :]), axis=2)
                    roi_2c_ede = roi_2c_ede.ravel()
                    roi_2c_ede = roi_2c_ede.reshape(-1, 64, 64, 2)
#                     print(roi_2c_ede.shape)

                else:
#                     print(i)
                    a = np.stack((xt[x_ede[0][i-1], :, :], xt[x_ede[0][i], :, :]), axis=2)
                    a = a.ravel()
                    a = a.reshape(-1, 64, 64, 2)
                    roi_2c_ede = np.vstack((roi_2c_ede, a))
#                     print(roi_2c_ede.shape)
                c2 += 1
    
    label_valid_nor = np.zeros(roi_2c.shape[0])
    label_valid_ede = np.ones(roi_2c_ede.shape[0])
    
    roi_2c_valid = np.vstack((roi_2c, roi_2c_ede))
    label_valid = np.hstack((label_valid_nor, label_valid_ede))
    
    
    print(roi_2c_train.shape)
    print(label_train.shape)
    print(roi_2c_valid.shape)
    print(label_valid.shape)
                                
    np.save('../data/final/3d/xtrain_40-2c-%d.npy' % (n), roi_2c_train)
    np.save('../data/final/3d/ytrain_40-2c-%d.npy' % (n), label_train)
    np.save('../data/final/3d/xvalid_10-2c-%d.npy' % (n), roi_2c_valid)
    np.save('../data/final/3d/yvalid_10-2c-%d.npy'% (n), label_valid)

(2011, 64, 64, 2)
(2011,)
(537, 64, 64, 2)
(537,)
(1982, 64, 64, 2)
(1982,)
(566, 64, 64, 2)
(566,)
(2054, 64, 64, 2)
(2054,)
(494, 64, 64, 2)
(494,)
(1958, 64, 64, 2)
(1958,)
(590, 64, 64, 2)
(590,)
(2030, 64, 64, 2)
(2030,)
(518, 64, 64, 2)
(518,)
(2064, 64, 64, 2)
(2064,)
(484, 64, 64, 2)
(484,)
(2031, 64, 64, 2)
(2031,)
(517, 64, 64, 2)
(517,)
(2046, 64, 64, 2)
(2046,)
(502, 64, 64, 2)
(502,)
(2019, 64, 64, 2)
(2019,)
(529, 64, 64, 2)
(529,)
(2037, 64, 64, 2)
(2037,)
(511, 64, 64, 2)
(511,)


## Create Train(50)Set

In [3]:
# data preprocessing 

ls_counts = 0
labels = np.zeros(5000)  # store labels

l = [i+1 for i in range(50)]
    # Made Simple Train Set
    # folder and file path
    # 40개 case를 trainset 으로 저장.
for j in l:
    if j < 10:
        j = '0' + str(j) + '/'
    else:
        j = str(j) + '/'

    path = '/media/miruware/data/train50_final/' + j

    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.jpg'):
                img = cv2.imread(path + filename)

                file_name, file_ext = os.path.splitext(filename)
#                     print(file_name)

#                 if filename.endswith('.hytag'):
                with open(path + file_name + '.hytag', 'r') as f:                  
                    while True:
                        line = f.readline()
                        if line: 
                            line = line.strip().split(',')
#                                 print(line)

#                                 ROI coordinate extraction
                            hytag_list = line
                            x=int(hytag_list[4]); y=int(hytag_list[5]); w=int(hytag_list[6]); h=int(hytag_list[7]) 

                            # label extraction
                            if hytag_list[0] == 'LBUID_636802211114365653_0529': # Edema
                                labels[ls_counts] = 1
                            elif hytag_list[0] == 'LBUID_636802210898995628_5475': # Normal 
                                labels[ls_counts] = 0

                            # Resize image(Minimal size for xline and yline)
                            roi = img[y:y+h, x:x+w]
                            roi_resize = cv2.resize(roi, (64, 64))
                            roi_resize_ravel = roi_resize.ravel()
                            roi_resize_ravel = roi_resize_ravel.reshape(-1, 64, 64, 3)

#                             roi_resize_ravel = rgb2gray(roi_resize_ravel)

                            ls_counts += 1

                            # stack arrays for making trainset
                            if ls_counts < 2:
                                roi_all = roi_resize_ravel
                            else:
                                roi_all = np.vstack((roi_all, roi_resize_ravel))                        
                        else:
                            break
#                 else:
#                     pass

labels = labels[:roi_all.shape[0]]

np.save('../data/final/3d/xtrain.npy', roi_all)
np.save('../data/final/3d/ytrain.npy', labels)

In [4]:
print(roi_all.shape)
print(labels.shape)

(2897, 64, 64, 3)
(2897,)


## Create TestSet

In [5]:
# data preprocessing 

ls_counts = 0
labels = np.zeros(5000)  # store labels

l = [i+1 for i in range(50, 73)]
    # Made Simple Train Set
    # folder and file path
    # 40개 case를 trainset 으로 저장.
for j in l:
    if j < 10:
        j = '0' + str(j) + '/'
    else:
        j = str(j) + '/'

    path = '/media/miruware/data/test23_final/' + j

    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.jpg'):
                img = cv2.imread(path + filename)

                file_name, file_ext = os.path.splitext(filename)
#                     print(file_name)

#                 if filename.endswith('.hytag'):
                with open(path + file_name + '.hytag', 'r') as f:                  
                    while True:
                        line = f.readline()
                        if line: 
                            line = line.strip().split(',')
#                                 print(line)

#                                 ROI coordinate extraction
                            hytag_list = line
                            x=int(hytag_list[4]); y=int(hytag_list[5]); w=int(hytag_list[6]); h=int(hytag_list[7]) 

                            # label extraction
                            if hytag_list[0] == 'LBUID_636802211114365653_0529': # Edema
                                labels[ls_counts] = 1
                            elif hytag_list[0] == 'LBUID_636802210898995628_5475': # Normal 
                                labels[ls_counts] = 0

                            # Resize image(Minimal size for xline and yline)
                            roi = img[y:y+h, x:x+w]
                            roi_resize = cv2.resize(roi, (64, 64))
                            roi_resize_ravel = roi_resize.ravel()
                            roi_resize_ravel = roi_resize_ravel.reshape(-1, 64, 64, 3)

#                             roi_resize_ravel = rgb2gray(roi_resize_ravel)

                            ls_counts += 1

                            # stack arrays for making trainset
                            if ls_counts < 2:
                                roi_all = roi_resize_ravel
                            else:
                                roi_all = np.vstack((roi_all, roi_resize_ravel))                        
                        else:
                            break
#                 else:
#                     pass

labels = labels[:roi_all.shape[0]]

print(roi_all.shape)
print(labels.shape)

np.save('../data/final/3d/xtest.npy', roi_all)
np.save('../data/final/3d/ytest.npy', labels)

(1297, 64, 64, 3)
(1297,)
